In [3]:
%run '/home/christianl/Zhang-Lab/Zhang Lab Code/Boilerplate_datahandling/Remote boilerplate/model_predictions.py'

Loading data...
Data shapes: x_train (12748, 1197), y_train (12748, 16100)

Loading MLR model...


/home/christianl/miniconda3/envs/remote_training/lib/python3.12/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/home/christianl/miniconda3/envs/remote_training/lib/python3.12/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


MLR predictions: (12748, 16100)

Loading XGBRF models...
XGBRF predictions: (12748, 16100)

Loading RNN model...
LOADING MODEL - EXACT TRAINING SCRIPT SEQUENCE

1. Loading checkpoint from: /home/christianl/Zhang-Lab/Zhang Lab Data/Saved models/RNN/uncentered_data_RNN/signaling_model.v1.pt

2. Loading network from: /home/christianl/Zhang-Lab/Zhang Lab Data/Full data files/network(full).tsv
   Network shape: (1153904, 3)
   Network columns: ['TF', 'Gene', 'Interaction']

3. Formatting network...

4. Using EXACT benchmark.py parameters
   projection_amplitude_in: 1.2
   projection_amplitude_out: 1.2
   bionet_params: {'target_steps': 150, 'max_steps': 10, 'exp_factor': 50, 'tolerance': 1e-20, 'leak': 0.01}

5. Initializing model with DataFrames...
   Input X_in shape: (12748, 1197)
   Input y_out shape: (12748, 16100)
  Filtered X_in: 1197 → 1197 features
  Filtered y_out: 16100 → 16100 features
   ✓ Model initialized (data automatically filtered)

6. Converting DataFrames to tensors...
 

In [7]:
%run '/home/christianl/Zhang-Lab/Zhang Lab Code/Boilerplate_datahandling/Remote boilerplate/model_load.py'

Loading pre-computed predictions...
✓ Loaded predictions for 3 models
  Training samples: 12748, Genes: 16100
  Test samples: 3187, Genes: 16100

✓ All functions loaded. Ready for analysis!


In [ ]:
############### UNIT TEST ###############

# Check what your y_test columns actually are:
print("y_test columns type:", type(y_train.columns))
print("First 5 column names:", y_train.columns[:5].tolist())

# Also check if y_test is actually a DataFrame or numpy array:
print("y_test type:", type(y_train))

In [ ]:
############### DIAGNOSTICS ###############

# diagnostics of MLR r2 scores for fitting to training set 

# checking to ensure predictions and groundtruth are comparable (they do)
assert y_train.shape == mlr_y_pred_train.shape

# compare r2 values
# mlr_loaded.score: 0.9041830139739396 on training data (fit)
print("mlr_loaded.score:", mlr_loaded.score(x_train, y_train))

# compute_metrics() is the flattened r2, where each gene's r2 is treated equally (flattened) before aggregating
# compute_metrics r2: 0.9323357932034118

# compute_metrics r2 > .score() r2 -> figured out that .score() when unspecified does uniform-average, different to compute_metrics r2 
# which calculates the variance-weighted r2, taking into account the individual variances of each gene.
# given the difference of 0.365 between the two, positive correlation between variance and R2 indicates that model performs better
# on genes with higher variance (more distinct expression patterns) than lower variance ones (ie. housekeeping, "silenced", etc.)

metrics_flat_mlr_train = compute_metrics(y_train.values, mlr_y_pred_train)
print("compute global R²:", metrics_flat_mlr_train['r2'])

mlr_train_agg_R2 = metrics_flat_mlr_train['r2']
mlr_train_agg_Pearsons = metrics_flat_mlr_train['pearson_r']

print(f"compute global R²: {mlr_train_agg_R2}")
print(f"compute global Pearson's R: {mlr_train_agg_Pearsons}")

# compute_metrics_per_gene() looks at the indiviudal r2 at per-gene resolution (using DFs to maintain biological relevance of each column) 

#compute_metrics_per_gene r2: 0        0.861767
#1        0.747973
#2        0.373796
#3        0.769920
#4        0.859807
#           ...   
#16095    0.736619
#16096    0.866326
#16097    0.924248
#16098    0.912513
#16099    0.900036
#Name: r2, Length: 16100, dtype: float64

metrics_flat_per_gene_mlr_train = compute_metrics_per_gene_test(y_train, mlr_y_pred_train)

mlr_train_pergene_R2 = metrics_flat_per_gene_mlr_train['r2']
mlr_train_pergene_Pearsons = metrics_flat_per_gene_mlr_train['[pearson_r]']

print("compute_metrics_per_gene R²:", mlr_train_pergene_R2)
print("computer_metrics_per_gene Pearson's R", mlr_train_pergene_Pearsons)

mlr_loaded.score: 0.9041830139739396


In [ ]:
############### DIAGNOSTICS ###############

# same diagnostics for XGBRF.v3 (trained on same x_train)
# checking to ensure predictions and groundtruth are comparable
assert y_train.shape == xgbrf_y_pred_train.shape

# computing XGBRF metrics (aggregate and per gene (per model is this case))
# compute global R²: 0.9136
metrics_flat_xgbrf_train = compute_metrics(y_train.values, xgbrf_y_pred_train)

xgbrf_train_agg_R2 = metrics_flat_xgbrf_train['r2']
xgbrf_train_agg_Pearsons = metrics_flat_xgbrf_train['pearson_r']

print(f"compute global R²: {xgbrf_train_agg_R2}")
print(f"compute global Pearson's R: {xgbrf_train_agg_Pearsons}")

#compute_metrics_per_gene R²: 0        0.798614
#1        0.732539
#2        0.390864
#3        0.717345
#4        0.819955
#           ...   
#16095    0.677608
#16096    0.801420
#16097    0.883455
#16098    0.861673
#16099    0.863954
#Name: r2, Length: 16100, dtype: float64

metrics_flat_per_gene_xgbrf_train = compute_metrics_per_gene_test(y_train, xgbrf_y_pred_train)

xgbrf_train_pergene_R2 = metrics_flat_per_gene_xgbrf_train['r2']
xgbrf_train_pergene_Pearsons = metrics_flat_per_gene_xgbrf_train['[pearson_r]']


print("compute_metrics_per_gene R²:", xgbrf_train_pergene_R2)
print("computer_metrics_per_gene Pearson's R", xgbrf_train_pergene_Pearsons)


In [ ]:
############### DIAGNOSTICS ###############


# same diagnostics for RNN.v1 (used as a reference for data preprocessing of other models )
# checking to ensure predictions and groundtruth are comparable
assert y_train.shape == rnn_y_pred_train.shape

# computing RNN metrics (aggregate and per gene (per model is this case))
# compute global R²: 0.7366
metrics_flat_rnn_train = compute_metrics(y_train.values, rnn_y_pred_train)

rnn_train_agg_R2 = metrics_flat_rnn_train['r2']
rnn_train_agg_Pearsons = metrics_flat_rnn_train['pearson_r']

print(f"compute global R²: {xgbrf_train_agg_R2}")
print(f"compute global Pearson's R: {xgbrf_train_agg_Pearsons}")
#compute_metrics_per_gene R²: 0        0.249111
#1       -0.058234
#2       -0.235948
#3        0.382344
#4        0.671084
#           ...   
#16095    0.336766
#16096    0.509802
#16097    0.657901
#16098    0.514623
#16099    0.622254
#Name: r2, Length: 16100, dtype: float64

metrics_flat_per_gene_rnn_train = compute_metrics_per_gene_test(y_train, rnn_y_pred_train)

rnn_train_pergene_R2 = metrics_flat_per_gene_rnn_train['r2']
rnn_train_pergene_Pearsons = metrics_flat_per_gene_rnn_train['[pearson_r]']

print("compute_metrics_per_gene R²:", rnn_train_pergene_R2)
print("computer_metrics_per_gene Pearson's R", rnn_train_pergene_Pearsons)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from scipy import stats as scipy_stats
import warnings
warnings.filterwarnings('ignore')

# Import your existing configuration
import sys
sys.path.append('/home/christianl/Zhang-Lab/Zhang Lab Code/Boilerplate_datahandling/Remote boilerplate')
from Fig_config_utilities import (
    MODEL_COLORS, DPI, FIGSIZE_SINGLE, FIGSIZE_WIDE,
    set_publication_style, predictions_train, y_train
)

# Set publication style
set_publication_style()

###############################################################################
# FIGURE 1: Aggregate Pearson's R with 95% CI
###############################################################################

def compute_aggregate_pearson_with_ci(y_true, y_pred, n_bootstrap=1000):
    """
    Compute aggregate Pearson's R across all prediction tasks with 95% CI.
    
    Parameters:
    -----------
    y_true : numpy array (n_samples, n_genes)
        Ground truth gene expression
    y_pred : numpy array (n_samples, n_genes)
        Predicted gene expression
    n_bootstrap : int
        Number of bootstrap iterations for CI estimation
        
    Returns:
    --------
    dict with 'pearson_r', 'ci_lower', 'ci_upper'
    """
    # Flatten all predictions for aggregate correlation
    y_true_flat = y_true.ravel()
    y_pred_flat = y_pred.ravel()
    
    # Overall Pearson's R
    pearson_r, _ = pearsonr(y_true_flat, y_pred_flat)
    
    # Bootstrap for 95% CI
    bootstrap_rs = []
    n_total = len(y_true_flat)
    
    for _ in range(n_bootstrap):
        # Sample with replacement
        indices = np.random.choice(n_total, size=n_total, replace=True)
        r_boot, _ = pearsonr(y_true_flat[indices], y_pred_flat[indices])
        bootstrap_rs.append(r_boot)
    
    # 95% CI
    ci_lower = np.percentile(bootstrap_rs, 2.5)
    ci_upper = np.percentile(bootstrap_rs, 97.5)
    
    return {
        'pearson_r': pearson_r,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'ci_range': ci_upper - ci_lower
    }


def plot_aggregate_pearson_comparison(predictions_dict, y_true, save_path=None):
    """
    Create bar plot comparing aggregate Pearson's R across models with 95% CI.
    
    Parameters:
    -----------
    predictions_dict : dict
        Dictionary with model names as keys and prediction arrays as values
    y_true : DataFrame or numpy array
        Ground truth values
    save_path : str, optional
        Path to save the figure
    """
    # Convert y_true to numpy if needed
    if isinstance(y_true, pd.DataFrame):
        y_true_array = y_true.values
    else:
        y_true_array = y_true
    
    # Compute metrics for each model
    results = {}
    for model_name, y_pred in predictions_dict.items():
        results[model_name] = compute_aggregate_pearson_with_ci(y_true_array, y_pred)
    
    # Prepare data for plotting
    models = list(results.keys())
    pearson_rs = [results[m]['pearson_r'] for m in models]
    ci_lowers = [results[m]['ci_lower'] for m in models]
    ci_uppers = [results[m]['ci_upper'] for m in models]
    errors_lower = [results[m]['pearson_r'] - results[m]['ci_lower'] for m in models]
    errors_upper = [results[m]['ci_upper'] - results[m]['pearson_r'] for m in models]
    
    # Create figure
    fig, ax = plt.subplots(figsize=FIGSIZE_SINGLE)
    
    # Bar positions
    x_pos = np.arange(len(models))
    
    # Create bars with error bars
    bars = ax.bar(x_pos, pearson_rs, 
                  color=[MODEL_COLORS.get(m, '#888888') for m in models],
                  alpha=0.8, edgecolor='black', linewidth=1.5)
    
    # Add error bars (95% CI)
    ax.errorbar(x_pos, pearson_rs, 
                yerr=[errors_lower, errors_upper],
                fmt='none', color='black', capsize=8, capthick=2, 
                linewidth=2, label='95% CI')
    
    # Customize plot
    ax.set_ylabel("Pearson's R (Aggregate)", fontsize=14, fontweight='bold')
    ax.set_xlabel("Model Architecture", fontsize=14, fontweight='bold')
    ax.set_title("Model Performance on Training Data\n(Aggregate across 16,100 genes)", 
                 fontsize=15, fontweight='bold', pad=20)
    ax.set_xticks(x_pos)
    ax.set_xticklabels(models, fontsize=12)
    ax.set_ylim([0, 1.05])
    
    # Add grid for readability
    ax.yaxis.grid(True, alpha=0.3, linestyle='--')
    ax.set_axisbelow(True)
    
    # Add value labels on bars
    for i, (bar, r_val, ci_low, ci_high) in enumerate(zip(bars, pearson_rs, ci_lowers, ci_uppers)):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                f'{r_val:.4f}\n[{ci_low:.4f}, {ci_high:.4f}]',
                ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=DPI, bbox_inches='tight')
        print(f"Figure saved to: {save_path}")
    
    return fig, ax, results


###############################################################################
# FIGURE 2: Pearson's R by CV bins
###############################################################################

def compute_cv_per_gene(y_true):
    """
    Compute coefficient of variation (CV) for each gene.
    
    Parameters:
    -----------
    y_true : numpy array (n_samples, n_genes)
        Ground truth gene expression
        
    Returns:
    --------
    numpy array of CV values (one per gene)
    """
    means = np.mean(y_true, axis=0)
    stds = np.std(y_true, axis=0)
    
    # Avoid division by zero
    cv = np.zeros_like(means)
    mask = means > 1e-10
    cv[mask] = (stds[mask] / means[mask]) * 100  # Express as percentage
    
    return cv


def compute_pearson_per_gene(y_true, y_pred):
    """
    Compute Pearson's R for each gene individually.
    
    Parameters:
    -----------
    y_true : numpy array (n_samples, n_genes)
    y_pred : numpy array (n_samples, n_genes)
        
    Returns:
    --------
    numpy array of Pearson's R values (one per gene)
    """
    n_genes = y_true.shape[1]
    pearson_rs = np.zeros(n_genes)
    
    for i in range(n_genes):
        if np.var(y_true[:, i]) > 1e-10:
            r, _ = pearsonr(y_true[:, i], y_pred[:, i])
            pearson_rs[i] = r
        else:
            pearson_rs[i] = np.nan
    
    return pearson_rs


def create_cv_bins(cv_values, bin_edges=None):
    """
    Create CV bins for grouping genes.
    
    Parameters:
    -----------
    cv_values : numpy array
        CV values for each gene
    bin_edges : list, optional
        Custom bin edges. If None, creates bins from 0-5%, 5-10%, ..., 20-25%
        
    Returns:
    --------
    bin_labels, bin_assignments
    """
    if bin_edges is None:
        # Default: 0-5%, 5-10%, 10-15%, 15-20%, 20-25%
        bin_edges = [0, 5, 10, 15, 20, 25]
    
    bin_labels = [f"{bin_edges[i]}-{bin_edges[i+1]}%" for i in range(len(bin_edges)-1)]
    bin_assignments = np.digitize(cv_values, bin_edges) - 1
    
    # Handle values outside bins
    bin_assignments = np.clip(bin_assignments, 0, len(bin_labels)-1)
    
    return bin_labels, bin_assignments


def plot_pearson_by_cv_bins(predictions_dict, y_true, bin_edges=None, save_path=None):
    """
    Create grouped bar plot of Pearson's R by CV bins for each model.
    
    Parameters:
    -----------
    predictions_dict : dict
        Dictionary with model names as keys and prediction arrays as values
    y_true : DataFrame or numpy array
        Ground truth values
    bin_edges : list, optional
        Custom CV bin edges
    save_path : str, optional
        Path to save the figure
    """
    # Convert y_true to numpy if needed
    if isinstance(y_true, pd.DataFrame):
        y_true_array = y_true.values
    else:
        y_true_array = y_true
    
    # Compute CV for each gene
    cv_values = compute_cv_per_gene(y_true_array)
    
    # Create bins
    bin_labels, bin_assignments = create_cv_bins(cv_values, bin_edges)
    n_bins = len(bin_labels)
    
    # Compute per-gene Pearson's R for each model
    model_results = {}
    for model_name, y_pred in predictions_dict.items():
        pearson_per_gene = compute_pearson_per_gene(y_true_array, y_pred)
        model_results[model_name] = pearson_per_gene
    
    # Aggregate by CV bins
    bin_data = {model: {bin_idx: [] for bin_idx in range(n_bins)} 
                for model in predictions_dict.keys()}
    
    for gene_idx in range(len(cv_values)):
        bin_idx = bin_assignments[gene_idx]
        for model_name, pearson_vals in model_results.items():
            if not np.isnan(pearson_vals[gene_idx]):
                bin_data[model_name][bin_idx].append(pearson_vals[gene_idx])
    
    # Calculate mean and SEM for each bin and model
    bin_means = {model: [] for model in predictions_dict.keys()}
    bin_sems = {model: [] for model in predictions_dict.keys()}
    
    for model_name in predictions_dict.keys():
        for bin_idx in range(n_bins):
            values = bin_data[model_name][bin_idx]
            if len(values) > 0:
                bin_means[model_name].append(np.mean(values))
                bin_sems[model_name].append(scipy_stats.sem(values))
            else:
                bin_means[model_name].append(0)
                bin_sems[model_name].append(0)
    
    # Create figure
    fig, ax = plt.subplots(figsize=FIGSIZE_WIDE)
    
    # Bar positions
    x = np.arange(n_bins)
    width = 0.25
    models = list(predictions_dict.keys())
    
    # Create grouped bars
    for i, model_name in enumerate(models):
        offset = (i - len(models)/2 + 0.5) * width
        bars = ax.bar(x + offset, bin_means[model_name], width,
                      label=model_name,
                      color=MODEL_COLORS.get(model_name, '#888888'),
                      alpha=0.8, edgecolor='black', linewidth=1.2)
        
        # Add error bars (SEM)
        ax.errorbar(x + offset, bin_means[model_name], 
                   yerr=bin_sems[model_name],
                   fmt='none', color='black', capsize=4, 
                   linewidth=1.5, alpha=0.7)
    
    # Customize plot
    ax.set_ylabel("Pearson's R (per-gene mean)", fontsize=14, fontweight='bold')
    ax.set_xlabel("Coefficient of Variation (CV) Bins", fontsize=14, fontweight='bold')
    ax.set_title("Model Performance Across Gene Expression Variability\n(Training Data)", 
                 fontsize=15, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(bin_labels, fontsize=11)
    ax.set_ylim([0, 1.05])
    ax.legend(loc='upper right', fontsize=12, framealpha=0.9)
    
    # Add grid
    ax.yaxis.grid(True, alpha=0.3, linestyle='--')
    ax.set_axisbelow(True)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=DPI, bbox_inches='tight')
        print(f"Figure saved to: {save_path}")
    
    # Print summary statistics
    print("\n" + "="*70)
    print("Summary: Mean Pearson's R by CV Bin")
    print("="*70)
    for model_name in models:
        print(f"\n{model_name}:")
        for i, bin_label in enumerate(bin_labels):
            n_genes_in_bin = len(bin_data[model_name][i])
            print(f"  {bin_label}: R={bin_means[model_name][i]:.4f} "
                  f"± {bin_sems[model_name][i]:.4f} (n={n_genes_in_bin} genes)")
    print("="*70 + "\n")
    
    return fig, ax, bin_data, bin_means, bin_sems


###############################################################################
# MAIN EXECUTION
###############################################################################

if __name__ == "__main__":
    print("Creating model fitness visualizations...")
    print(f"Using {len(predictions_train)} models")
    print(f"Training set size: {y_train.shape}")
    
    # Figure 1: Aggregate Pearson's R with 95% CI
    print("\n" + "="*70)
    print("FIGURE 1: Aggregate Pearson's R Comparison")
    print("="*70)
    fig1, ax1, results1 = plot_aggregate_pearson_comparison(
        predictions_train, 
        y_train,
        save_path='/mnt/user-data/outputs/Fig1_aggregate_pearson_comparison.png'
    )
    
    # Print results
    for model_name, metrics in results1.items():
        print(f"\n{model_name}:")
        print(f"  Pearson's R: {metrics['pearson_r']:.6f}")
        print(f"  95% CI: [{metrics['ci_lower']:.6f}, {metrics['ci_upper']:.6f}]")
        print(f"  CI Range: {metrics['ci_range']:.6f}")
    
    # Figure 2: Pearson's R by CV bins
    print("\n" + "="*70)
    print("FIGURE 2: Pearson's R by CV Bins")
    print("="*70)
    fig2, ax2, bin_data, bin_means, bin_sems = plot_pearson_by_cv_bins(
        predictions_train,
        y_train,
        bin_edges=[0, 5, 10, 15, 20, 25],
        save_path='/mnt/user-data/outputs/Fig2_pearson_by_cv_bins.png'
    )
    
    print("\nVisualization complete! Check the output directory for figures.")
    plt.show()